In [1]:
from pathlib import Path
import os
import simplejson as json
import polars as pl
import json_repair
from tqdm import tqdm
from more_itertools import chunked
from loguru import logger
import asyncio as asio
from justatom.tooling.stl import reuuid

from justatom.etc.io import io_snapshot
from justatom.tooling.reqs import openai_chat
from justatom.tooling.coro import _limit_concurrency
from justatom.running.igni import IGNIRunner

/Users/justatom/ISpell/envs/justatom/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def wrapper_for_props(d: dict, must_include_keys: list[str] = None) -> dict:
    """
    :param d: Source doc
    :param must_include_keys: List of keys to include
    :return: New doc with only specified `must_include_keys`
    """
    must_include_keys = d.keys() if must_include_keys is None else must_include_keys
    return {key: d[key] for key in must_include_keys if key in d}

In [3]:
def source_from_dataset(dataset_name_or_path, **props):
    from justatom.storing.dataset import API as DatasetApi
    import polars as pl

    maybe_df_or_iter = DatasetApi.named(dataset_name_or_path).iterator(**props)
    if isinstance(maybe_df_or_iter, pl.DataFrame):
        pl_data = maybe_df_or_iter
    else:
        dataset = list(maybe_df_or_iter)
        pl_data = pl.from_dicts(dataset)
    return pl_data

In [4]:
pl_docs = source_from_dataset(Path(os.getcwd()) / ".data" / "polaroids.ai.data.json")
js_docs = pl_docs.to_dicts()

In [5]:
js_docs[0]

{'content': 'Внутри располагалась первая красивая вещь, которую я встречала в 13 Дистрикте: копия луга, заполненного настоящими деревьями и цветущими растениями, и в изобилии порхающими колибри. Битти сидел неподвижно в инвалидном кресле посреди луга, наблюдая за нежно-зелёной птичкой, зависшей в воздухе и пьющей нектар из большого цветка апельсинового дерева.',
 'title': 'Сойка-пересмешница',
 'author': 'Сьюзен Коллинз',
 'type': 'book',
 'has_image': False,
 'img_path': None,
 'speaker': None,
 'queries': ['Как выглядел луг в Дистрикте 13?',
  'Какие элементы природы были представлены в копии луга из 13-го Дистрикта во вселенной "Сойка-пересмешница"?'],
 'translation': "Inside was the first beautiful thing I'd seen in District 13: a replica of a meadow, filled with real trees and blooming plants, and plenty of hummingbirds fluttering about. Beetee was sitting motionless in a wheelchair in the middle of the meadow, watching a tender green bird hovering in the air and drinking nectar f

In [6]:
async def pipeline(
    js_docs: list[dict],
    pr_runner,
    openai_model_name: str,
    batch_size: int = 16,
    coros_size: int = 2,
    save_snapshot_every: int = 5,
    snapshot_prefix: str = None,
    snapshot_where: str = None,
    timeout: int = 512,
    must_include_keys: list[str] | None = None,
    validate_json_response: bool = False,
):
    """
    We process `js_docs` by chunks where each chunk is of size `batch_size`.
    Each chunk is processed asynchronously via parallel `coros_size` coroutines.

    :param js_docs: documents to process
    :param pr_runner: One of the instance `IPromptRunner` to create specific prompt
    """
    pipes = []

    for i, batch in tqdm(enumerate(chunked(js_docs, n=batch_size))):
        _batch = batch
        cur_result = await asio.gather(
            *_limit_concurrency(
                [
                    openai_chat(
                        pr_runner.prompt(**d),
                        timeout=timeout,
                        model=openai_model_name,
                        props=wrapper_for_props(d, must_include_keys=must_include_keys),
                    )
                    for d in _batch
                ],
                concurrency=coros_size,
            )
        )
        if not validate_json_response:
            pipes.extend(cur_result)
        else:
            # NOTE: Order of execution is preserved.
            js_answer_docs = [
                pr_runner.finalize(
                    raw_response=js_res["response"], **wrapper_for_props(js_doc, must_include_keys=must_include_keys)
                )
                for js_doc, js_res in zip(batch, cur_result, strict=True)
            ]
            pipes.extend(js_answer_docs)

        if (i + 1) % save_snapshot_every == 0:
            io_snapshot(pipes, where=snapshot_where, snapshot_number=str(i + 1), snapshot_prefix=snapshot_prefix)
    return pipes

In [7]:
system_prompt = f"""
Ты - эксперт, специализирующийся на объяснении трудной из параграфа так, чтобы поняли даже дети.
Твоя задача выделить ключевые слова (вместе с объяснением их контекста) из предоставленного тебе параграфа. Параграф может быть из разных областей, таких как: наука, история, биология, религия, общие факты, книги, игры или фильмы.
"""

openai_model_name = "gpt-4-turbo"
batch_size = 4
coros_size = 2
snapshot_every_iters = 1
must_include_keys = ["chunk_id", "content"]
snapshot_prefix = "KWARGS"
snapshot_where = "outputs"
source_language="русском"
timeout  = 512

In [8]:
pr_runner = await IGNIRunner.KEYWORDER(
    system_prompt=system_prompt,
    source_language=source_language
)

In [9]:
pr_runner

In [10]:
pr_runner.prompt(content="мои поздравления", title="Книги - игры - фильмы")

[{'role': 'system',
  'content': 'Ты - эксперт, специализирующийся на объяснении трудной из параграфа так, чтобы поняли даже дети.\nТвоя задача выделить ключевые слова (вместе с объяснением их контекста) из предоставленного тебе параграфа. Параграф может быть из разных областей, таких как: наука, история, биология, религия, общие факты, книги, игры или фильмы.'},
 {'role': 'user',
  'content': 'Обрати внимание, что ключевые слова или фразы должны быть подстрокой параграфа и состоять из НЕ более двух, максимум трех слов.\n\n        Каждая фраза или ключевое слово должны иметь краткое, но емкое объяснение на русском языке в зависимости от контекста, в котором они употреблены.\xa0\n\n        Параграф из вселенной "Книги - игры - фильмы":\nмои поздравления\n\n\n        Выдай ответ в виде  json в формате: {"keywords_or_phrases": [{"keyword_or_phrase": <Выделенная тобою фраза>, "explanation": <Объяснение на русском языке для ребенка в соответствии с контекстом, в котором употреблена ключевая

In [11]:
js_response = await pipeline(
    js_docs[:10],
    pr_runner=pr_runner,
    openai_model_name=openai_model_name,
    batch_size=batch_size,
    coros_size=coros_size,
    timeout=timeout,
    save_snapshot_every=snapshot_every_iters,
    snapshot_prefix=snapshot_prefix,
    snapshot_where=snapshot_where,
    must_include_keys=must_include_keys
)

0it [00:00, ?it/s]

3it [01:16, 25.39s/it]


In [12]:
js_response_docs_parsed = [
    {
        **json_repair.loads(js_doc['response']),
        **wrapper_for_props(js_doc, must_include_keys=["chunk_id", "content"])
    } for js_doc in js_response
]

In [13]:
js_response_docs_parsed[0]

{'keywords_or_phrases': [{'keyword_or_phrase': '13 Дистрикт',
   'explanation': 'Это одно из разделений территории в вымышленном мире, где происходят действия книги.'},
  {'keyword_or_phrase': 'копия луга',
   'explanation': 'Искусственно созданное место, которое выглядит как настоящий луг с деревьями и цветами.'},
  {'keyword_or_phrase': 'колибри',
   'explanation': 'Маленькая птица, известная своей способностью парить в воздухе, пьящая нектар из цветов.'},
  {'keyword_or_phrase': 'инвалидное кресло',
   'explanation': 'Специальное кресло на колёсах, которое используется людьми, у которых есть проблемы с передвижением.'},
  {'keyword_or_phrase': 'нектар',
   'explanation': 'Сладкая жидкость, которую производят цветы и пьют птицы, как колибри, для питания.'},
  {'keyword_or_phrase': 'апельсинового дерева',
   'explanation': 'Дерево, на котором растут апельсины, цветок этого дерева также производит нектар.'}],
 'chunk_id': '50275823-ca7a-50f2-ae1f-3f945d24ab47',
 'content': 'Внутри расп

In [14]:
pl_response_docs = pl.from_dicts(js_response_docs_parsed)

In [15]:
logger.info(f"There are N=[{pl_response_docs.shape[0]}] docs with `keywords_or_phrases`")
pl_response_docs.head()

2025-01-03 19:27:46.275 | INFO     | __main__:<module>:1 - There are N=[10] docs with `keywords_or_phrases`


keywords_or_phrases,chunk_id,content
list[struct[2]],str,str
"[{""13 Дистрикт"",""Это одно из разделений территории в вымышленном мире, где происходят действия книги.""}, {""копия луга"",""Искусственно созданное место, которое выглядит как настоящий луг с деревьями и цветами.""}, … {""апельсинового дерева"",""Дерево, на котором растут апельсины, цветок этого дерева также производит нектар.""}]","""50275823-ca7a-…","""Внутри распола…"
"[{""просто псих"",""Это выражение говорит о том, что герой считается необычным или странным человеком другими людьми.""}, {""их принципы"",""Здесь говорится о правилах или моральных убеждениях, которым люди следуют.""}, … {""Я не чудовище"",""Герой говорит, что он не злой и страшный, как его могут воспринимать другие.""}]","""0835dee2-f0be-…","""Для них ты про…"
"[{""желчный, раздражительный, злой"",""Раскольников проснулся в плохом настроении, он был очень злой и раздражён.""}, {""крошечная клетушка"",""Очень маленькая, тесная комната, напоминающая клетку для животного.""}, … {""мономанов"",""Люди, настолько сосредоточенные или одержимые чем-то одним, что это занимает все их мысли и восприятие.""}]","""8b2538de-db7b-…","""Он проснулся н…"
"[{""Шай-Хулуд"",""Шай-Хулуд - это величайшее создание, огромный песчаный червь в мире 'Дюна', который считается священным.""}, {""следуй за мной"",""Это приглашение идти за кем-то, чтобы показать более легкий или безопасный путь.""}, … {""волна радости"",""Выражение используется для описания сильного и внезапного чувства счастья.""}]","""2c967240-5ed5-…","""И вот она пере…"
"[{""Судный день"",""Время крупной катастрофы, когда люди отправляются в места убежища.""}, {""машинистам"",""Водители поездов в метро.""}, … {""жителями метро"",""Люди, которые живут в подземке после катастрофы.""}]","""fafce8be-6d5e-…","""Инструкция о т…"
